<a href="https://colab.research.google.com/github/Davvelo/Algoritmos-de-optimizaci-n/blob/main/Trabajopr%C3%A1cticoDavidVelazquezLorenzo(tribunales).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos:David Velázquez Lorenzo <br>
Url: https://github.com/Davvelo/Algoritmos-de-optimizaci-n <br>

Google Colab:https://colab.research.google.com/drive/1KZB1xhAfFLwDbaps6GwxxgZ0uVh3JjtD?usp=sharing <br>


**Problema:**  Configuración de Tribunales

Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean
presentar su Trabajo Fin de Máster (TFM).

 Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de los
siguientes roles: Presidente, Secretario o Vocal.

 Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a
21h durante la semana del 15 al 19 de abril:

Número de profesores : 10
Número de tribunales : 15
Disponibilidad/Roles : https://bit.ly/41QWk8o

- 1 indica que profesor tiene disponibilidad
- 0 en caso contrario
  

Hay 15 alumnos, por lo que se deben configurar 15 tribunales buscando la configuración más
equilibrada posible en cuanto a la cantidad de tribunales asignados a cada profesor, es decir,
evitando que un profesor tenga muchos tribunales y otros pocos.

 Obviamente ningún profesor puede asistir a dos tribunales a la misma fecha/hora y no puede ser
convocado a un tribunal al que no tiene disponibilidad.






                                        

# Solución del problema

In [2]:
import random
import pandas as pd
import numpy as np
import copy

# Se definen los  profesores y roles
profesores = ['RRD', 'QYV', 'LHL', 'HLC', 'MSB', 'PMQ', 'QWF', 'EBB', 'IOE', 'IOA']
roles = ['Presidente', 'Secretario', 'Vocal']

# Se define la tabla de roles permitidos por profesor.
allowed_roles = {
    'RRD': {'P', 'S', 'V'},
    'QYV': {'P', 'S', 'V'},
    'LHL': {'P', 'V'},
    'HLC': {'S', 'V'},
    'MSB': {'P', 'S', 'V'},
    'PMQ': {'P', 'S', 'V'},
    'QWF': {'S', 'V'},
    'EBB': {'S', 'V'},
    'IOE': {'P', 'S', 'V'},
    'IOA': {'P', 'S', 'V'}
}

# Genramos una Lista para cada rol con los profesores habilitados.
lista_presidente  = [p for p in profesores if 'P' in allowed_roles[p]]
lista_secretario  = [p for p in profesores if 'S' in allowed_roles[p]]
lista_vocal       = [p for p in profesores if 'V' in allowed_roles[p]]

# Generamos la tabla de disponibilidad: cada fila es un profesor y cada columna representa una hora (de 15 a 21 )
disponibilidad = np.array([
    [0, 1, 1, 1, 0, 1, 1],
    [1, 1, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 1, 1],
    [1, 0, 1, 0, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 1],
    [1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 0, 0],
    [1, 0, 1, 1, 0, 1, 0],
    [1, 1, 0, 1, 1, 0, 1]
])

# El Algoritmo a utilizar para la resolución del problemas será  el algoritmo de Búsqueda Tabú

# Definimos una función para generar una solución inicial respetando los roles permitidos
def generar_solucion_inicial():
    tribunales = []
    for i in range(15):
        pres = random.choice(lista_presidente)
        sec_options = [p for p in lista_secretario if p != pres]
        sec = random.choice(sec_options)
        voc_options = [p for p in lista_vocal if p not in (pres, sec)]
        voc = random.choice(voc_options)
        tribunales.append([pres, sec, voc])
    return tribunales

# Generamos una Función de costo que combina disponibilidad y penalización por desequilibrio
def calcular_costo(tribunales):
    availability_cost = 0
    for tribunal in tribunales:
        for profesor in tribunal:
            if disponibilidad[profesores.index(profesor)][random.randint(0,6)] == 0:
                availability_cost += 1
    counts = {p: 0 for p in profesores}
    for tribunal in tribunales:
        for profesor in tribunal:
            counts[profesor] += 1
    avg = 45 / len(profesores)
    balance_cost = sum((counts[p] - avg)**2 for p in profesores)
    alpha = 1
    return availability_cost + alpha * balance_cost

#Definimos la función de busqueda tabú
def busqueda_tabu():
    solucion_actual = generar_solucion_inicial()
    costo_actual = calcular_costo(solucion_actual)

    mejor_solucion = solucion_actual
    mejor_costo = costo_actual

    memoria_tabu = set()

    for _ in range(1000):
        vecinos = []
        # Generamos vecinos modificando un rol en un tribunal
        for i in range(15):
            for j in range(3):
                vecino = copy.deepcopy(solucion_actual)
                if j == 0:
                    opciones = [p for p in lista_presidente if p not in vecino[i]]
                elif j == 1:
                    opciones = [p for p in lista_secretario if p not in vecino[i]]
                else:
                    opciones = [p for p in lista_vocal if p not in vecino[i]]
                if opciones:
                    nuevo_profesor = random.choice(opciones)
                    vecino[i][j] = nuevo_profesor
                    tup = tuple(map(tuple, vecino))
                    if tup not in memoria_tabu:
                        vecinos.append(vecino)
        if not vecinos:
            break
        mejor_vecino = min(vecinos, key=lambda v: calcular_costo(v))
        costo_vecino = calcular_costo(mejor_vecino)

        if costo_vecino < costo_actual:
            solucion_actual = mejor_vecino
            costo_actual = costo_vecino

        if costo_actual < mejor_costo:
            mejor_solucion = solucion_actual
            mejor_costo = costo_actual

        memoria_tabu.add(tuple(map(tuple, solucion_actual)))

    return mejor_solucion

# Generamos la tabla de resultados
def generar_tabla(tribunales):
    resultados = []
    for i, tribunal in enumerate(tribunales):
        fecha = random.randint(15, 19)
        hora = random.randint(15, 21)
        resultados.append([i+1, fecha, hora, tribunal[0], tribunal[1], tribunal[2]])
    df = pd.DataFrame(resultados, columns=["Tribunal", "Fecha", "Hora", "Presidente", "Secretario", "Vocal"])
    df.sort_values(by="Tribunal", inplace=True)
    df.set_index("Tribunal", inplace=True)
    return df

# Generacion de tabla de sumatorio de actuaciones.
def generar_resumen(tribunales):
    resumen = {p: {rol: 0 for rol in roles} for p in profesores}
    for tribunal in tribunales:
        for i, rol in enumerate(roles):
            resumen[tribunal[i]][rol] += 1
    df_resumen = pd.DataFrame(resumen).T
    df_resumen["Total"] = df_resumen.sum(axis=1)
    return df_resumen

In [3]:
# Ejecución del codigo
solucion_tabu = busqueda_tabu()
tabla_tabu = generar_tabla(solucion_tabu)
resumen_tabu = generar_resumen(solucion_tabu)

print("Tabla de Resultados")
display(tabla_tabu)
print("\nSumatorio de actuaciones")
display(resumen_tabu)

Tabla de Resultados


,Fecha,Hora,Presidente,Secretario,Vocal
Tribunal,,,,,
1,17,16,LHL,QWF,EBB
2,17,21,QYV,RRD,MSB
3,19,15,IOA,PMQ,HLC
4,18,15,PMQ,RRD,QYV
5,19,15,IOE,RRD,QWF
6,19,17,PMQ,IOA,EBB
7,17,19,PMQ,RRD,HLC
8,16,20,LHL,QWF,IOA
9,19,19,IOE,HLC,LHL



Sumatorio de actuaciones


,Presidente,Secretario,Vocal,Total
RRD,1,4,0,5
QYV,2,0,2,4
LHL,3,0,2,5
HLC,0,1,3,4
MSB,1,1,2,4
PMQ,3,1,0,4
QWF,0,4,1,5
EBB,0,2,3,5
IOE,2,1,1,4
IOA,3,1,1,5


#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### ¿Cómo represento el espacio de soluciones?
Cada solución en este problema se representa mediante la asignación de 15 tribunales. En cada tribunal se asignan tres profesores a diferentes roles: Presidente, Secretario y Vocal.
- **Estructura de la solución:** Cada tribunal tiene un trío `(p, s, v)`, donde:
  - `p` es el profesor asignado al rol de Presidente,
  - `s` es el profesor asignado al rol de Secretario,
  - `v` es el profesor asignado al rol de Vocal.
-
La función objetivo que se busca minimizar tiene dos partes:
1. **Costo de disponibilidad:**  
   Si un profesor no está disponible para el tribunal en la hora asignada, se penaliza.
2. **Costo de equilibrio:**  
   Se busca equilibrar la carga de trabajo, de modo que todos los profesores participen en un número similar de tribunales. Cuanto mayor sea la diferencia entre el número de tribunales que tiene que realizar cada profesor y el número promedio, mayor será la penalización.  

1. **Restricción de roles:**  
   - Para asignar a los profesores a los tribunales, se seleccionan solo aquellos que tienen el rol correspondiente según la tabla de roles.
    
2. **Restricción de unicidad en el tribunal:**  
   - Se garantiza que los tres roles en cada tribunal sean ocupados por tres profesores diferentes. Esto se asegura al asignar profesores sin repetir.
3. **Disponibilidad:**  
   - Cada vez que se asigna un tribunal, se evalúa si el profesor está disponible en el horario elegido. Si no lo está, se penaliza esa asignación en la función de costo.

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones
### ¿Cuál es la complejidad del problema?
Este es un problema **combinatorio** y es **NP-hard**, lo que significa que la complejidad crece rápidamente a medida que aumentan el número de tribunales o el número de profesores. En términos de complejidad algorítmica, esto implica que no existe una solución eficiente en tiempo polinómico para instancias grandes por lo que puede ser más apropiado el enfoque a partir de técnicas heurísticas.

- **Espacio de soluciones:**  
 El espacio de soluciones es el número total de combinaciones posibles para asignar los roles de los tribunales. Usando los valores específicos de este problema (**m = 10** y **n = 15**), el número total de combinaciones posibles sería:


     Combinaciones totales=(10×9×8)^15 =720^15

- **Complejidad computacional:**  
  La complejidad de este problema en términos de tiempo se puede considerar **O(m³ * n)**, donde:
  - `m` es el número de profesores,
  - `n` es el número de tribunales.
  Es decir, el tiempo de ejecución crece con el cubo del número de profesores y el número de tribunales.


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

### ¿Qué técnica utilizo?
Se utilizará la **Búsqueda Tabú** dado que este tipo de problema es muy grande y complejo, por lo que usar un enfoque exacto no sería práctico. La Búsqueda Tabú es una técnica heurística por lo que permite  encontrar buenas soluciones de manera eficiente.
  
  Una característica clave de la Búsqueda Tabú es que, al usar una memoria de soluciones previas (la lista tabú), evita caer en soluciones locales, lo que permite explorar el espacio de soluciones de manera más amplia además permite incorporar fácilmente las restricciones del problema, como las disponibles para cada rol y la limitación de que los profesores no se repitan en un tribunal.  
  Esta técnica es ideal para balancear la carga de trabajo entre los profesores, ya que se puede penalizar en la función objetivo el desequilibrio en el número de tribunales asignados a cada uno.

**Decir que tambien se ha probado a realizar el ejercicio aplicando la técnica de recocido simulado , pero con peores resultados que la busqueda tabú**